In [1]:
# Install
!pip install -q datasets
!pip install -q transformers
!pip install -q evaluate
!pip install -q rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==

In [2]:
# Import

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from evaluate import load as load_metric
import torch
# Log in to Hugging Face
from google.colab import userdata
from huggingface_hub import login
my_hugging_face_token = userdata.get('huggingface_token')
login(token=my_hugging_face_token)

In [5]:
# load models
teacher_model_id = "meta-llama/Llama-3.1-8B-instruct"
student_model_id = "Jiexing1028/llama-3-8b-workout"
dataset_id = "Jiexing1028/workout-plan"
test_size = 0.2
max_input_length = 512
max_gen_length = 128

# Load original dataset (train only)
test_dataset = load_dataset(dataset_id, split="test")
# Shuffle the dataset and select the first 100 samples
test_subset = test_dataset.shuffle(seed=42).select(range(100))

# Load models and tokenizer
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_id)
student_tokenizer = AutoTokenizer.from_pretrained(student_model_id)

teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_id, torch_dtype=torch.float16).cuda()
student_model = AutoModelForCausalLM.from_pretrained(student_model_id, torch_dtype=torch.float16).cuda()

teacher_pipeline = pipeline("text-generation", model=teacher_model, tokenizer=teacher_tokenizer, device=0)
student_pipeline = pipeline("text-generation", model=student_model, tokenizer=student_tokenizer, device=0)


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Device set to use cuda:0
Device set to use cuda:0


In [6]:
# Load evaluation metrics
f1_metric = load_metric("f1")
bleu_metric = load_metric("bleu")
exact_match_metric = load_metric("exact_match")

# Format prompt for your task
def format_prompt(example):
    return f"Question: {example['question']}\nAnswer:"

# Generate answers
def generate_answers(model_pipeline, inputs):
    outputs = model_pipeline(inputs, max_new_tokens=max_gen_length, do_sample=False)
    return [o[0]["generated_text"].replace(i, "").strip() for o, i in zip(outputs, inputs)]

# Get prompts and references
prompts = [format_prompt(x) for x in test_subset]
references = test_subset["answer"]
bleu_refs = [[ref] for ref in references]





In [ ]:
# Generate teacher predictions
print("Generating with teacher...")
teacher_preds = generate_answers(teacher_pipeline, prompts)

# Generate student prediction
print("Generating with student...")
student_preds = generate_answers(student_pipeline, prompts)

In [9]:
rouge_metric = load_metric("rouge")
# Evaluation function
def evaluate(preds, refs):
    # ROUGE metric computation
    # ROUGE metric computation
    rouge_scores = rouge_metric.compute(predictions=preds, references=refs)

    # Accessing the ROUGE-L F1 score directly from the output
    rougeL = rouge_scores["rougeL"]

    return {
        "rougeL": rougeL
    }

# Evaluate both models



print("\n--- Evaluation Results ---")

# Print results 1B instruct
student_scores = evaluate(student_preds, references)
print("Student Model:")
print(student_scores)

# Print results 3B instruct

teacher_scores = evaluate(teacher_preds, references)
print("Teacher Model:")
print(teacher_scores)



--- Evaluation Results ---


# New section

In [ ]:
# 3B fine tuned - 0.1496
# 3B inst - 0.1444
# 8B fine tuned - 0.1521
# 8B inst - 0.1478

In [2]:
# Log in to Hugging Face
from google.colab import userdata
from huggingface_hub import login
my_hugging_face_token = userdata.get('huggingface_token')
login(token=my_hugging_face_token)

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
import evaluate
max_gen_length = 256

# Load models and tokenizer
teacher_model_id = "meta-llama/Llama-3.1-8B-instruct"
student_model_id = "Jiexing1028/llama-3-8b-workout"
dataset_id = "Jiexing1028/workout-plan"
test_size = 0.2

print("Generating with teacher...")
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_id)
teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_id, torch_dtype=torch.float16).cuda()
teacher_pipeline = pipeline("text-generation", model=teacher_model, tokenizer=teacher_tokenizer, device=0)

print("Generating with student...")
student_tokenizer = AutoTokenizer.from_pretrained(student_model_id)
student_model = AutoModelForCausalLM.from_pretrained(student_model_id, torch_dtype=torch.float16).cuda()
student_pipeline = pipeline("text-generation", model=student_model, tokenizer=student_tokenizer, device=0)


Generating with teacher...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:

# Load evaluation metrics using the 'evaluate' library
f1_metric = evaluate.load("f1")
bleu_metric = evaluate.load("bleu")
exact_match_metric = evaluate.load("exact_match")
rouge_metric = evaluate.load("rouge")

# Load dataset
test_dataset = load_dataset(dataset_id, split="test")
test_subset = test_dataset.shuffle(seed=42).select(range(50))

# Format prompt for your task
def format_prompt(example):
    return f"Question: {example['question']}\nAnswer:"

# Generate answers
def generate_answers(model_pipeline, inputs):
    outputs = model_pipeline(inputs, max_new_tokens=max_gen_length, do_sample=False)
    return [o[0]["generated_text"].replace(i, "").strip() for o, i in zip(outputs, inputs)]

# Get prompts and references
prompts = [format_prompt(x) for x in test_subset]
references = test_subset["answer"]
bleu_refs = [[ref] for ref in references]

# Loss and Perplexity Function
def evaluate_qa_perplexity(model, tokenizer, questions, answers, max_samples=100, max_length=512, device='cuda'):
    model.eval()
    model.to(device)

    total_loss = 0
    total_tokens = 0

    for q, a in zip(questions[:max_samples], answers[:max_samples]):
        prompt = f"Question: {q}\nAnswer:"
        full_input = prompt + " " + a

        inputs = tokenizer(full_input, return_tensors="pt", truncation=True, max_length=max_length)
        input_ids = inputs["input_ids"].to(device)

        # Mask the question (we only want to supervise the answer)
        answer_token_start = tokenizer(prompt, return_tensors="pt")["input_ids"].size(1)
        labels = input_ids.clone()
        labels[0, :answer_token_start] = -100

        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            num_tokens = (labels != -100).sum().item()
            total_loss += loss.item() * num_tokens
            total_tokens += num_tokens

    avg_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    return avg_loss, perplexity

# Evaluation function
def evaluate(preds, refs):
    rouge_scores = rouge_metric.compute(predictions=preds, references=refs)
    rougeL = rouge_scores["rougeL"]
    return {"rougeL": rougeL}

# Run Evaluation
print("\n--- Evaluation Results ---")

# Evaluate Teacher Model
teacher_preds = generate_answers(teacher_pipeline, prompts)
teacher_scores = evaluate(teacher_preds, references)
teacher_loss, teacher_ppl = evaluate_qa_perplexity(teacher_model, teacher_tokenizer, test_subset["question"], references)

print("Teacher Model:")
print(teacher_scores)
print(f"Loss: {teacher_loss:.4f}, Perplexity: {teacher_ppl:.2f}")

# Evaluate Student Model
student_preds = generate_answers(student_pipeline, prompts)
student_scores = evaluate(student_preds, references)
student_loss, student_ppl = evaluate_qa_perplexity(student_model, student_tokenizer, test_subset["question"], references)

print("Student Model:")
print(student_scores)
print(f"Loss: {student_loss:.4f}, Perplexity: {student_ppl:.2f}")



README.md:   0%|          | 0.00/415 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/280k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/932 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/233 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- Evaluation Results ---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Teacher Model:
{'rougeL': np.float64(0.1710885039543485)}
Loss: 0.8805, Perplexity: 2.41
Student Model:
{'rougeL': np.float64(0.17749327122383574)}
Loss: 0.7538, Perplexity: 2.13


In [ ]:
# Teacher Model: 8B inst
# {'rougeL': np.float64(0.1710885039543485)}
# Loss: 0.8805, Perplexity: 2.41
# Student Model: 8B distilled
# {'rougeL': np.float64(0.17749327122383574)}
# Loss: 0.7538, Perplexity: 2.13